### Instagram Data Extractor using Selenium !!

In this notebook we will use the power of python and selenium to extract instagram data. We will use this data to find out whether the person is most likely to follow back if I interact with him by liking his profile pictures and following his account. 

For data collection we will be following these steps: 

1. Enter Hash tag using hashtag file of src
2. Extract 300 user per hashtag
3. Check whether those users are in UsersData table. 
4. Extract for only those whose data is not in the table.
5. Recall back the userdata into dataframe, where date is today's date. 



In [1]:


# Change directory
import os
os.chdir('C:\\Users\\PC\\Instagram_Production\\V0.001')

from src.data import data
# Necessary imports
from datetime import datetime, timedelta
from itertools import dropwhile, takewhile
from instaloader import Instaloader, Profile
from src.insta import InstagramBot
from sklearn.externals.joblib import load
from src.FeatureExtraction import featureExtraction
import sqlite3 as db
import numpy as np
import pandas as pd
from numpy import random
from selenium.common.exceptions import TimeoutException



C:\Users\PC\Anaconda3\envs\Twitter\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'cp1252', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)
C:\Users\PC\Anaconda3\envs\Twitter\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


2020-08-19 09:59:50
2020-08-19 09:59:49
2020-08-19 09:59:49
2020-08-19 09:59:49
2020-08-19 09:59:48
2020-08-19 09:59:48
2020-08-19 09:59:47
2020-08-19 09:59:47
2020-08-19 09:59:46
2020-08-19 09:59:46
2020-08-19 09:59:46
2020-08-19 09:59:46
2020-08-19 09:59:46
2020-08-19 09:59:45
2020-08-19 09:59:44
2020-08-19 09:59:44
2020-08-19 09:59:43
2020-08-19 09:59:43
2020-08-19 09:59:43
2020-08-19 09:59:43
2020-08-19 09:59:43
2020-08-19 09:59:43
2020-08-19 09:59:42
2020-08-19 09:59:42
2020-08-19 09:59:42
2020-08-19 09:59:42
2020-08-19 09:59:42
2020-08-19 09:59:42
2020-08-19 09:59:41
2020-08-19 09:59:41
2020-08-19 09:59:41
2020-08-19 09:59:40
2020-08-19 09:59:40
2020-08-19 09:59:40
2020-08-19 09:59:40
2020-08-19 09:59:40
2020-08-19 09:59:39
2020-08-19 09:59:39
2020-08-19 09:59:38
2020-08-19 09:59:37
2020-08-19 09:59:35
2020-08-19 09:59:30
2020-08-19 09:58:03
2020-08-19 09:54:55
2020-08-19 09:54:49
2020-08-19 09:51:17
2020-08-19 07:24:40
2020-08-19 06:44:04


#### By selecting the following hashtag, it implies selecting the user who have liked the post having the following hashtag

In [ ]:
hashtags = ['photography', 'cute']

We will do the following stuff in the codes below: 
1. Extract 100 user per hashtag
2. Check whether those users are in UsersData table.
3. Extract for only those whose data is not in the table.
4. Recall back the userdata into dataframe, where date is today's date.

In [ ]:
try:
    t = random.randint(0, len(hashtags))
    hashtag = hashtags[t]

    # Step 2
    L = Instaloader()
    data = []
    posts = L.get_hashtag_posts(hashtag)
    likes = set()

    today = datetime.today()
    tomorrow = datetime.today() + timedelta(days = 2)
    yesterday = datetime.today() + timedelta(days = -1)

    # Since is big | less past 
    SINCE = tomorrow
    # Until is small | past
    UNTIL = yesterday

    # Extract the list of people to send request to, note : We are selecting the users who are recently active. 

    for post in takewhile(lambda p: p.date > UNTIL, dropwhile(lambda p: p.date > SINCE, posts)):
        print(post.date)
        likes = post.get_likes()
        for like in likes:
            data.append(like.username)
        users = np.unique(data)
        if len(users) >= 100: 
            break;
        else:
            pass

    # Step 3
    # Connect to a database (or create one if it doesn't exist) for saving the list in database
    conn = db.connect('data/sqldb/Ig_Data.db')
    # Create a 'cursor' for executing commands
    c = conn.cursor()

    query_db = ''' SELECT * FROM UsersData'''
    usersDB = pd.read_sql(query_db, conn)

    alreadySaved = usersDB['username'].values

    toCheck = []
    for user in users:
        if user in alreadySaved:
            pass
        else: 
            toCheck.append(user)

    del alreadySaved
    del usersDB
    del data
    del likes

    # Step 4
    ## Login using my account and checking for profile information
    username = 'Enter your username'
    password = 'Enter your password'

    insta = InstagramBot(username, password)
    insta.login()
    # Step 5: Extract user data
    for user in toCheck[:100]:
        user, AccountType, posts, following, followers = insta.accountPrivacy(user)
        query_insert = '''INSERT INTO UsersData VALUES ({}{}{},{}{}{},{}{}{},{},{},{})'''\
                    .format("'", today.date() ,"'", "'", user, "'","'", AccountType,"'", posts, following, followers)
        c.execute(query_insert)
        conn.commit()

    # Step 6: Save data into SQL database!
    query = '''SELECT * FROM UsersData WHERE date = {}{}{}'''.format("'", today.date(), "'")
    df = pd.read_sql(query, conn)
    # assert len(df) == len(toCheck)
except TimeoutException:
    pass